# Neural Factorization Machines

In [ ]:
import tensorflow as tf
from sequence_feature_layer import SequenceFeatures
from tensorflow import feature_column as fc
from tensorflow.keras.layers import Layer, DenseFeatures, Dense, LayerNormalization, Dropout, Embedding, Conv1D, BatchNormalization

## 0.准备工作

In [ ]:
seq_1 = fc.sequence_categorical_column_with_hash_bucket('seq_1', hash_bucket_size=10, dtype=tf.int64)
seq_2 = fc.sequence_categorical_column_with_hash_bucket('seq_2', hash_bucket_size=10, dtype=tf.int64)
target = fc.sequence_categorical_column_with_hash_bucket('target', hash_bucket_size=10, dtype=tf.int64)

seq_1_col = fc.embedding_column(seq_1, dimension=8)
seq_2_col = fc.embedding_column(seq_2, dimension=8)

target_col = fc.embedding_column(target, dimension=8)
columns = [seq_1_col, seq_2_col, target_col]
features={
  "seq_1": tf.sparse.SparseTensor(
      indices=[[0, 0], [0, 1], [1, 0], [1, 1], [2, 0]],
      values=[1100, 1101, 1102, 1101, 1103],
      dense_shape=[3, 2]),
  "seq_2": tf.sparse.SparseTensor(
      indices=[[0, 0], [0, 1], [1, 0], [1, 1], [2, 0]],
      values=[2200, 2201, 2202, 2201, 2203],
      dense_shape=[3, 2]),
  "target": tf.sparse.SparseTensor(
      indices=[[0, 0],[1,0],[2,0]],
      values=[1102,1103,1100],
      dense_shape=[3, 1]),

}
tf.sparse.to_dense(features['seq_1'])

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1100, 1101],
       [1102, 1101],
       [1103,    0]], dtype=int32)>

In [ ]:
sequence_feature_layer = SequenceFeatures(columns, name='sequence_features_input_layer')
seq_emb_dict, seq_len_dict = sequence_feature_layer(features)
seq_emb_dict.keys()

dict_keys(['seq_1_embedding', 'seq_2_embedding', 'target_embedding'])

In [ ]:
x = tf.concat(list(seq_emb_dict.values()), axis=1)
x

<tf.Tensor: shape=(3, 5, 8), dtype=float32, numpy=
array([[[ 0.01905249,  0.2711436 , -0.16974296, -0.41354162,
         -0.45203206, -0.1237737 ,  0.01610972, -0.3776071 ],
        [ 0.5781513 ,  0.60547346, -0.01465757, -0.36699656,
         -0.040447  ,  0.29422325,  0.35566035, -0.03534933],
        [-0.40586925, -0.642621  , -0.17732742, -0.05926026,
         -0.18070647, -0.11091012,  0.10545808, -0.42049518],
        [ 0.50875366, -0.3321689 ,  0.40166482, -0.0013006 ,
         -0.04791581, -0.18354316, -0.33573154,  0.11226024],
        [ 0.3518665 ,  0.24142267, -0.59557533,  0.02514912,
         -0.03666467,  0.13006938, -0.07243787, -0.21522275]],

       [[-0.2970747 , -0.22957708, -0.17061137,  0.08516189,
          0.5246019 ,  0.43462422, -0.3932593 , -0.07194421],
        [ 0.5781513 ,  0.60547346, -0.01465757, -0.36699656,
         -0.040447  ,  0.29422325,  0.35566035, -0.03534933],
        [-0.22881229, -0.53723127, -0.09205798,  0.33169273,
         -0.39137065, -0.

## 1. Bi-Interaction

In [ ]:
net = 0.5 * (tf.pow(tf.reduce_sum(x, axis=1), 2) - tf.reduce_sum(tf.pow(x, 2), axis=1))
net

<tf.Tensor: shape=(3, 8), dtype=float32, numpy=
array([[ 0.11230832, -0.5005896 , -0.13389155,  0.1779626 ,  0.16597268,
        -0.08237883, -0.12553449,  0.24864766],
       [-0.0264926 , -0.36298501, -0.07339802, -0.10777485, -0.22062041,
        -0.1933475 , -0.10560042, -0.15279365],
       [-0.08256596, -0.14059371, -0.03335603, -0.02521746,  0.2638645 ,
        -0.01124423,  0.12509727,  0.36167687]], dtype=float32)>

In [ ]:
bn = BatchNormalization()
hidden_units = [16, 8]
dnn = [Dense(unit, activation='relu') for unit in hidden_units]
output_layer = Dense(1, activation='sigmoid')

## 1.1计算

In [ ]:
net = bn(net)
for nn in dnn:
  net = nn(net)
output_layer(net)

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[0.4864927 ],
       [0.48034126],
       [0.4964484 ]], dtype=float32)>